In [1]:
# Boltzmann Machines

# Importing the libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [2]:
def read_data(L):
    """
    :type L: int(system size)
    :rtype train,test: pd.dataframe(with phases labeled)
    """
    train_filename='configuration'+str(L)+'.csv'
    test_filename='test'+str(L)+'.csv'
    columns=['temperature']+['Spin'+str(i) for i in xrange(1,L*L+1)]
    train=pd.read_csv('Ising_data/'+'L'+str(L)+'/'+train_filename,names=columns)
    test=pd.read_csv('Ising_data/'+'L'+str(L)+'/'+test_filename,names=columns)
    
    # decide critical temperature
    filename='Cv'+str(L)+'.csv'
    specific_heat=pd.read_csv('Ising_data/'+'L'+str(L)+'/'+filename,names=['temperature','Cv'])
    Tc=specific_heat['temperature'][np.argmax(specific_heat['Cv'])]
    
    # add phase column
    train['FM']=[int(T<=Tc) for T in train['temperature']]
    test['FM']=[int(T<=Tc) for T in test['temperature']]
    train['PM']=[int(T>Tc) for T in train['temperature']]
    test['PM']=[int(T>Tc) for T in test['temperature']]
    
    return train, test

In [3]:
train, test=read_data(40)

In [4]:
def data_process(train, test):
    """
    :type train,test: pd.dataframe(with phases labeled)
    :rtype trX, trY, teX, teY
    """
    # shuffle
    train=train.sample(frac=1).reset_index(drop=True)
    
    trX, trY = train.drop(['FM','PM','temperature'],axis=1), np.array(train[['FM','PM']])
    teX, teY = test.drop(['FM','PM','temperature'],axis=1), np.array(test[['FM','PM']])
    
    return trX.as_matrix().tolist(), trY.tolist(), teX.as_matrix().tolist(), teY.tolist()

In [5]:
trX, _, teX, _=data_process(train, test)

In [6]:
trX = torch.FloatTensor(trX)
teX = torch.FloatTensor(teX)

**Creating the architecture of the Neural Network**:

In [7]:
class RBM():
    # self.W(torch tensor (nh, nv)): weights
    # self.a(torch tensor (1, nh)): hidden bias
    # self.b(torch tensor (1, nv)): input bias
    def __init__(self, nv, nh):
        """
        nv(int): number of visible nodes
        nh(int): number of hidden nodes
        """
        self.W = torch.randn(nh, nv) # initialize weights
        self.a = torch.randn(1, nh) # initialize hidden bias
        self.b = torch.randn(1, nv) # initialize input bias
    
    # probability: p(h|v) = sigmoid(W*x + a) used for Gibbs sampler
    def sample_h(self, x):
        """
        x(int): visible neurons
        return variables:
        p_h_given_v(float): probability of hidden nodes given visible nodes
        torch.bernoulli(p_h_given_v): whether a hidden node is activated or not
        """
        wx = torch.mm(x, self.W.t())
        activation = wx + self.a.expand_as(wx) # make sure bias is applied to each line of the mini-batch
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)
    # probability: p(v|h) = sigmoid(W*y + a) used for Gibbs sampler
    def sample_v(self, y):
        """
        y(int): hidden neurons
        return variables:
        p_h_given_v(float): probability of hidden nodes given visible nodes
        torch.bernoulli(p_h_given_v): whether a hidden node is activated or not
        """
        wy = torch.mm(y, self.W)
        activation = wy + self.b.expand_as(wy) # make sure bias is applied to each line of the mini-batch
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    def train(self, v0, vk, ph0, phk):
        self.W += torch.mm(v0.t(), ph0)-torch.mm(vk.t(), phk)
        self.b += torch.sum((v0 - vk),0)
        self.a += torch.sum((ph0 - phk),0)

In [9]:
nv = len(trX[0])
nh = 100
batch_size = 100
rbm = RBM(nv, nh)

**Training the RBM**

In [ ]:
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.